## Visualizing Instacart Data

requirements for queries: 
* at least 1 aggregate function per query
* 3 joins
* 4 group by
* 2 having
* 4 order by 

used so far:
* 3/3 joins idk check 3rd query
* 3/4 group by idk check 3rd query
* 1/2 having
* 3/4 order by idk check 3rd query

## QUERIES

#### Query 1


From this query, we wanted to extract the most "loyal" customers (i.e. users that have used instacart the most). 
The query shows that there are a lot of loyal customers. 

In [16]:
%%bigquery
SELECT user_id, COUNT(order_id) AS frequency
FROM instacart_modeled.Orders
GROUP BY user_id
ORDER BY frequency DESC
LIMIT 1375

,user_id,frequency
0,17997,100
1,6398,100
2,41509,100
3,9437,100
4,53488,100
...,...,...
1370,66600,100
1371,58853,100
1372,81342,100
1373,199280,100


#### Query 2

Following the same logic as the previous query, exactly 1,374 customers have ordered a total of 100 times, 99 customers a total of 47 times, etc.

In [17]:
%%bigquery
SELECT frequency AS frequency_ordered, COUNT(frequency) AS total_customers
FROM (select user_id, COUNT(order_id) AS frequency
FROM instacart_modeled.Orders
GROUP BY user_id)
GROUP BY frequency_ordered
ORDER BY frequency_ordered DESC

,frequency_ordered,total_customers
0,100,1374
1,99,47
2,98,50
3,97,54
4,96,67
...,...,...
92,8,11700
93,7,13850
94,6,16165
95,5,19590


We see the max order frequency is 100.

In [18]:
%%bigquery
SELECT MAX(frequency_ordered) AS max_order_freq
FROM
    (SELECT frequency AS frequency_ordered, COUNT(frequency) AS total_customers
    FROM 
        (SELECT user_id, COUNT(order_id) AS frequency
        FROM instacart_modeled.Orders
        GROUP BY user_id)
    GROUP BY frequency_ordered
    ORDER BY frequency_ordered DESC)

,max_order_freq
0,100


#### Query 3

Analyze the orders of the most "loyal" customers and get the top 40 most popular items

In [28]:
%%bigquery
SELECT p.product_name AS Product, count(1) AS Frequency
FROM
  (SELECT os.order_id
  FROM
    (SELECT user_id, COUNT(order_id) as frequency
    FROM instacart_modeled.Orders
    GROUP BY user_id
    HAVING(frequency=100)) us
  RIGHT OUTER JOIN instacart_modeled.Orders os
  ON us.user_id=os.user_id) o
RIGHT OUTER JOIN instacart_modeled.Order_Products op
ON o.order_id=op.order_id
RIGHT OUTER JOIN instacart_modeled.Products p
ON op.product_id=p.product_id
GROUP BY p.product_name
ORDER BY frequency desc
LIMIT 40

,Product,Frequency
0,Banana,491291
1,Bag of Organic Bananas,394930
2,Organic Strawberries,275577
3,Organic Baby Spinach,251705
4,Organic Hass Avocado,220877
5,Organic Avocado,184224
6,Large Lemon,160792
7,Strawberries,149445
8,Limes,146660
9,Organic Whole Milk,142813


Checking the query above by looking at the occurrence of bananas.

In [29]:
%%bigquery
SELECT COUNT(1) AS frequency
FROM
(SELECT os.order_id
FROM
  (SELECT user_id, COUNT(order_id) AS frequency
  FROM instacart_modeled.Orders
  GROUP BY user_id
  HAVING(frequency=100)) us
RIGHT OUTER JOIN instacart_modeled.Orders os
ON us.user_id=os.user_id) o
RIGHT OUTER JOIN instacart_modeled.Order_Products op
ON o.order_id=op.order_id
RIGHT OUTER JOIN instacart_modeled.Products p
ON op.product_id=p.product_id
WHERE p.product_name='Banana'
ORDER BY frequency desc

,frequency
0,491291


#### Query 4

explanation

#### Query 5

explanation

#### Query 6

explanation

### VIEWS

#### Query 3 VIEW
Analyze the orders of the most "loyal" customers and get the top 10 most popular items

In [30]:
%%bigquery
CREATE OR REPLACE VIEW instacart_modeled.v_Loyal_Customers_Top_10_Products AS
    (SELECT p.product_name AS Product, count(1) AS Frequency
    FROM
      (SELECT os.order_id
      FROM
        (SELECT user_id, COUNT(order_id) as frequency
        FROM `responsive-cab-267123.instacart_modeled.Orders`
        GROUP BY user_id
        HAVING(frequency=100)) us
      RIGHT OUTER JOIN `responsive-cab-267123.instacart_modeled.Orders` os
      ON us.user_id=os.user_id) o
    RIGHT OUTER JOIN `responsive-cab-267123.instacart_modeled.Order_Products` op
    ON o.order_id=op.order_id
    RIGHT OUTER JOIN `responsive-cab-267123.instacart_modeled.Products` p
    ON op.product_id=p.product_id
    GROUP BY p.product_name
    ORDER BY frequency DESC
    LIMIT 10)

""


Checking correct making of view

In [31]:
%%bigquery
SELECT * 
FROM instacart_modeled.v_Loyal_Customers_Top_10_Products
ORDER BY frequency DESC
LIMIT 5

,Product,Frequency
0,Banana,491291
1,Bag of Organic Bananas,394930
2,Organic Strawberries,275577
3,Organic Baby Spinach,251705
4,Organic Hass Avocado,220877
